In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_list, struct, sort_array, avg, count

# --- Spark tối ưu ---
spark = (
    SparkSession.builder
        .appName("ETL_Gold_Fast")
        .config("spark.sql.shuffle.partitions", "8")   # giảm shuffle (mặc định 200 -> quá nặng)
        .config("spark.driver.memory", "4g")
        .config("spark.executor.memory", "4g")
        .config("parquet.enable.summary-metadata", "false")
        .getOrCreate()
)

base_path = "/home/jovyan/work"

print("🚀 [GOLD] Bắt đầu tổng hợp...")

# --- 1. Load dữ liệu Silver ---
df_otto = spark.read.parquet(f"{base_path}/data_lake/silver/otto")
df_amazon = spark.read.parquet(f"{base_path}/data_lake/silver/amazon")

# Cache nếu muốn tăng tốc
df_otto.cache()
df_amazon.cache()

# ============================================================
# 🥇 GOLD: DỮ LIỆU CHO AI TRAINING (không dùng Window)
# ============================================================

df_ai = (
    df_otto
    .groupBy("session")
    .agg(
        collect_list(struct("ts", "aid")).alias("tmp")
    )
    .withColumn("final_sequence", sort_array("tmp").getField("aid"))
    .drop("tmp")
)

df_ai.write.mode("overwrite").parquet(f"{base_path}/data_lake/gold/ai_training")


# ============================================================
# 🥇 GOLD: DỮ LIỆU CHO DASHBOARD / BI
# ============================================================

df_report = (
    df_amazon
    .groupBy("asin")
    .agg(
        count("*").alias("count"),
        avg("rating").alias("score")
    )
)

df_report.write.mode("overwrite").parquet(f"{base_path}/data_lake/gold/dashboard")


print("✅ [GOLD] Hoàn thành.")
spark.stop()


🚀 [GOLD] Bắt đầu tổng hợp...
✅ [GOLD] Hoàn thành.
